<a href="https://colab.research.google.com/github/Pavneet922/AI-Driven-Crop-Disease-Prediction-and-Management-System/blob/main/Train_plant_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pavneetk08","key":"e7d67d432b2214e941cb9a682296c7e1"}'}

In [6]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
import zipfile
zip_ref = zipfile.ZipFile('new-plant-diseases-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: 'new-plant-diseases-dataset.zip'

# Data Preprocessing
### Training Image Preprocessing

In [23]:
training_set = tf.keras.utils.image_dataset_from_directory(
    "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    labels = "inferred",
    label_mode = "categorical",
    class_names = None,
    color_mode = "rgb",
    batch_size = 32,
    image_size = (128, 128),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = "bilinear",
    follow_links = False,
    crop_to_aspect_ratio = False,
    data_format = None
)

Found 70295 files belonging to 38 classes.


### Validation Image Preprocessing

In [60]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
    labels = "inferred",
    label_mode = "categorical",
    class_names = None,
    color_mode = "rgb",
    batch_size = 32,
    image_size = (128, 128),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = "bilinear",
    follow_links = False,
    crop_to_aspect_ratio = False,
    data_format = None
)

Found 17572 files belonging to 38 classes.


In [25]:
training_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [29]:
for x, y in training_set:
  print(x, x.shape)
  print(y, y.shape)
  break

tf.Tensor(
[[[[ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   ...
   [138.25 127.25 133.25]
   [129.   118.   124.  ]
   [141.25 130.25 136.25]]

  [[ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   ...
   [145.25 134.25 140.25]
   [137.75 126.75 132.75]
   [137.5  126.5  132.5 ]]

  [[ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   [ 10.    13.    18.  ]
   ...
   [143.5  132.5  138.5 ]
   [131.5  120.5  126.5 ]
   [153.5  142.5  148.5 ]]

  ...

  [[169.75 166.75 173.75]
   [171.75 168.75 175.75]
   [166.   163.   170.  ]
   ...
   [177.25 172.25 179.25]
   [171.25 166.25 173.25]
   [171.75 166.75 173.75]]

  [[180.   177.   184.  ]
   [177.75 174.75 181.75]
   [169.5  166.5  173.5 ]
   ...
   [178.25 173.25 180.25]
   [172.   167.   174.  ]
   [172.25 167.25 174.25]]

  [[171.   168.   175.  ]
   [170.   167.   174.  ]
   [171.5  168.5  175.5 ]
   ...
   [179.75 174.75 181.75]
   [181.5  176.5  183.5 ]
   [175.25 170.25 17

#Building Model

In [70]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [104]:
model = Sequential()

##Building Convolution Layer

In [105]:
model.add(Conv2D(filters = 32, kernel_size=3, padding="same", activation="relu", input_shape=[128, 128, 3]))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [106]:
model.add(Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [107]:
model.add(Conv2D(filters = 128, kernel_size = 3, padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = 3, activation = "relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [108]:
model.add(Conv2D(filters = 256, kernel_size = 3, padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = 3, activation = "relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [109]:
model.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = 3, activation = "relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [110]:
model.add(Dropout(0.25)) #to avoid overshooting

In [111]:
model.add(Flatten())

In [112]:
model.add(Dense(units = 1500, activation = "relu"))

In [113]:
model.add(Dropout(0.4))

In [114]:
#Output Layer
model.add(Dense(units = 38, activation = "softmax"))

##Compiling Model

In [115]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss = "categorical_crossentropy", metrics =["accuracy"])

In [116]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_59 (Conv2D)                   │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_60 (Conv2D)                   │ (None, 126, 126, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_61 (Conv2D)                   │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_62 (Conv2D)                   │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_63 (Conv2D)                   │ (None, 30, 30, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_64 (Conv2D)                   │ (None, 28, 28, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_65 (Conv2D)                   │ (None, 14, 14, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_66 (Conv2D)                   │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_67 (Conv2D)                   │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_68 (Conv2D)                   │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1500)                │       3,073,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1500)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 38)                  │          57,038 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,842,762 (29.92 MB)

 Trainable params: 7,842,762 (29.92 MB)

 Non-trainable params: 0 (0.00 B)

##Model Training

In [10]:
training_history = model.fit(x = training_set, validation_data = validation_set, epochs = 10)

NameError: name 'model' is not defined

#Model Evaluation

In [ ]:
#Model evaluation on Training set
train_loss, train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss, train_acc)

In [ ]:
#Model evaluation on Training set
val_loss, val_acc = model.evaluate(validation_set)

In [ ]:
print(val_loss, val_acc)

#Saving Model

In [ ]:
model.save("trained_model.keras") #or "trained_model.h5" then the size will be very large. Keras is better

In [ ]:
training_history.history

In [ ]:
#Recording history in json
import json
with open("training_history.json", "w") as f:
  json.dump(training_history.history, f)

#Accuracy Visualization

In [ ]:
epochs = [i for i in range(1, 11)]
plt.plot(epochs, training_history.history["accuracy"], color = "red", label = "Training accuracy")
plt.plot(epochs, training_history.history["accuracy"], color = "blue", label = "Validation accuracy")
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy result")
plt.title("Visualization of accuracy result")
plt.legend()
plt.show()

##Some other matrix for model evaluation

In [ ]:
class_name = validation_set.class_names
class_name

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
    labels = "inferred",
    label_mode = "categorical",
    class_names = None,
    color_mode = "rgb",
    batch_size = 32,
    image_size = (128, 128),
    shuffle = False,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = "bilinear",
    follow_links = False,
    crop_to_aspect_ratio = False,
    data_format = None

In [ ]:
y_pred = model.predict(test_set)
y_pred, y_pred.shape

In [ ]:
predicted_categories = tf.argmax(y_pred, axis = 1)
predicted_categories

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis = 0)
true_categories

In [ ]:
y_true = tf.argmax(true_categories, axis = 1)
y_true

In [ ]:
from sklearn.metrics import classification_report
classification_report(y_true, predicted_categories, target_names = class_name)

In [ ]:
cm = confusion_matrix(y_true, predicted_categories)
cm

##Confusion Matrix Visualaziation

In [ ]:
plt.figure(figsize = (40, 40))
sns.heatmap(cm, annot = True )
plt.xlabel("Predicted class")
plt.ylabel("Actual class")
plt.title("Plant disease prediction confusion matrix")
plt.show()